In [12]:
import pandas as pd
from langchain import PromptTemplate
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI
import warnings
from langchain.chains import LLMChain
warnings.filterwarnings("ignore")

In [13]:
data = pd.read_csv("../Resources/Raw/air_pollution.csv")
print("Cantidad de registros:",str(data.shape[0]))
data.head(5)


Cantidad de registros: 6985


,city,country,2017,2018,2019,2020,2021,2022,2023
0,Kabul,Afghanistan,0,61.8,58.8,46.5,37.5,17.1,18.1
1,Tirana,Albania,0,0,0,16.0,12.5,14.5,14.4
2,Algiers,Algeria,0,0,21.2,20.2,20.0,17.8,17.4
3,Ordino,Andorra,0,0,0,7.4,7.3,5.4,5.3
4,Luanda,Angola,0,0,15.9,13.0,11.0,8.8,8.7


In [14]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model = AzureChatOpenAI(temperature=0, deployment_name="chat")

In [15]:
template = """
You have the following information {context} which contains information about the population in millions of each country and its corresponding city
and you must answer the following question {question}
Limit yourself to delivering the answer.
Response in spanish
"""

prompt = PromptTemplate(
    template = template,
    input_variables = ["context","question"]
)

llm = LLMChain(llm=model, prompt=prompt)

llm.run(context = data,
         question = "Que año se tuvo mayor cantidad de población? y con cuanto fue?")




'El año con la mayor cantidad de población fue el año 2017, con una población de 61.8 millones.'

In [16]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../Resources/Pdf/JavaScript.pdf")
pages = loader.load_and_split()

In [17]:
pages[:5]

[Document(page_content='EloquentJavaScript\n3raedición\nMarijn Haverbeke', metadata={'source': '../Resources/Pdf/JavaScript.pdf', 'page': 1}),
 Document(page_content='Copyright © 2018 por Marijn Haverbeke\nEste libro está bajo la licencia Creative Commons Atribución-NoComercial\n3.0 ( http://creativecommons.org/licenses/by-nc/3.0/ ). El código también\npuede considerarse bajo una licencia MIT ( http://opensource.org/licenses/\nMIT).\nLas ilustraciones han sido contribuidas por varios artistas: Cover and chap-\nter illustrations by Madalina Tantareanu. Pixel art in Chapters 7 and 16 by\nAntonio Perdomo Pastor. Regular expression diagrams in Chapter 9 gener-\nated with regexper.com by Jeff Avallone. Village photograph in Chapter 11 by\nFabrice Creuzot. Game concept for Chapter 15 by Thomas Palef .\nThe third edition of Eloquent JavaScript was made possible by 325 financial\nbackers .\nYou can buy a print version of this book, with an extra bonus chapter included,\nprinted by No Starch Pr

In [31]:
template = """
You are only handling the following information {context} and you must answer the following question {question}.
"""

prompt = PromptTemplate(
    template = template,
    input_variables = ["context","question"]
)
llm_js = LLMChain(llm=model, prompt=prompt)



In [30]:
llm_js.run(context = pages[:10],
         question = "Cómo puedo declarar una variable en javascript?")



> Entering new LLMChain chain...
Prompt after formatting:

You are only handling the following information [Document(page_content='EloquentJavaScript\n3raedición\nMarijn Haverbeke', metadata={'source': '../Resources/Pdf/JavaScript.pdf', 'page': 1}), Document(page_content='Copyright © 2018 por Marijn Haverbeke\nEste libro está bajo la licencia Creative Commons Atribución-NoComercial\n3.0 ( http://creativecommons.org/licenses/by-nc/3.0/ ). El código también\npuede considerarse bajo una licencia MIT ( http://opensource.org/licenses/\nMIT).\nLas ilustraciones han sido contribuidas por varios artistas: Cover and chap-\nter illustrations by Madalina Tantareanu. Pixel art in Chapters 7 and 16 by\nAntonio Perdomo Pastor. Regular expression diagrams in Chapter 9 gener-\nated with regexper.com by Jeff Avallone. Village photograph in Chapter 11 by\nFabrice Creuzot. Game concept for Chapter 15 by Thomas Palef .\nThe third edition of Eloquent JavaScript was made possible by 325 financial\nbackers

'Para declarar una variable en JavaScript, se utiliza la palabra clave "var", seguida del nombre de la variable y opcionalmente se le puede asignar un valor. Por ejemplo:\n\nvar nombreVariable;\nvar edad = 25;\n\nTambién se pueden declarar variables utilizando "let" y "const" en lugar de "var", dependiendo del alcance y la mutabilidad que se desee para la variable.'

In [37]:
# Opción 2
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.chains.question_answering import load_qa_chain
llm_qa_documents = load_qa_chain(
    model,
    chain_type="stuff",
    prompt=prompt
)

llm_qa_documents.run(input_documents = pages[:10], question = "Cómo puedo declarar una variable en javascript?")

'En JavaScript, puedes declarar una variable utilizando la palabra clave "var", "let" o "const". Por ejemplo:\n\nvar nombre = "Juan";\nlet edad = 25;\nconst PI = 3.1416;\n\nLa palabra clave "var" se utiliza para declarar una variable con alcance de función o global. La palabra clave "let" se utiliza para declarar una variable con alcance de bloque. La palabra clave "const" se utiliza para declarar una constante, cuyo valor no puede ser modificado una vez asignado.\n\nEspero que esta información te sea útil.'

In [39]:
import json
datos_list = []
with open('../Resources/Raw/netflix_titles.json', 'r') as archivo_json:
    for linea in archivo_json:
        datos = json.loads(linea)
        datos_list.append(datos)

In [42]:
datos_list[:1]

[{'show_id': 's1',
  'type': 'Movie',
  'title': 'Dick Johnson Is Dead',
  'director': 'Kirsten Johnson',
  'cast': None,
  'country': 'United States',
  'date_added': 'September 25, 2021',
  'release_year': 2020,
  'rating': 'PG-13',
  'duration': '90 min',
  'listed_in': 'Documentaries',
  'description': 'As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.'}]

In [43]:
template = """
You are only handling the following information {context} and you must answer the following question {question}.
"""

prompt = PromptTemplate(
    template = template,
    input_variables = ["context","question"]
)
llm_movies= LLMChain(llm=model, prompt=prompt)

In [48]:
llm_movies.run(context = datos_list[:55],
         question = "Que película o telenovela me recomiendas donde la protagonista sea mujer")

'Te recomendaría la película "Dick Johnson Is Dead". Es un documental en el que la directora Kirsten Johnson protagoniza la historia junto a su padre, explorando temas como la vida, la muerte y el amor familiar.'